In [1]:
import os

import cv2

import numpy as np

from matplotlib import pyplot as plt

In [2]:
def read_image(image_path, plot=False):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if plot:
        plt.imshow(img)
        plt.show()
    return img

def show_compare(images: list, image_names: list = []):
    count = len(images)
    count_names = len(image_names)
    fig, axes = plt.subplots(1, count)
    for cnt in range(count):
        axes[cnt].imshow(images[cnt])
        axes[cnt].axis('off')
        if cnt < count_names:
            axes[cnt].set_title(image_names[cnt])
    plt.show()

In [3]:
def scale(image):
    minimum = np.min(image)
    filtered = (255*(image - minimum)/np.ptp(image)).astype(np.uint8)
    return filtered

    
def clip(image):
    filtered = np.clip(image, 0, 255).astype(np.uint8)
    return filtered

def final_remake_to_image(image, kernel):
    sharpen = np.array([
        [0, -1, 0,],
        [-1, 5, -1,],
        [0, -1, 0,],
    ])
    if np.sum(kernel) == 0 or (kernel.shape == sharpen.shape and np.all(kernel  == sharpen)):
        print("Cliping array...")
        filtered = clip(image)
    else:
        print("Scaling array...")
        filtered = scale(image)
    return filtered


def apply_kernel_filter(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    kh, kw = kernel.shape
    H, W, C = image.shape

    pad_h = kernel.shape[0]//2
    pad_v = kernel.shape[1]//2

    image_with_kernel = image.copy()
    image_padded = np.pad(image, ((pad_h, pad_h), (pad_v, pad_v), (0, 0)), mode='edge')

    image_with_kernel = np.zeros_like(image, dtype=np.float32)

    for ch in range(C):
        windows = np.zeros((H, W, kh, kw), dtype=np.float32)
        for i in range(kh):
            for j in range(kw):
                windows[:, :, i, j] = image_padded[i:i+H, j:j+W, ch]

        image_with_kernel[..., ch] = np.sum(windows * kernel, axis=(2, 3))

    return final_remake_to_image(image_with_kernel, kernel)

# apply_kernel_filter(image_float, gaussian)

In [4]:
image_paths = ["../data/" + path for path in os.listdir("../data")]
plt.rcParams['figure.figsize'] = [15, 10]

In [5]:
# visible_diff = ["../data/1344829.png", "../data/1350497.png", "../data/dark.png", "../data/kodim01.png", "../data/kodim05.jpg", "../data/kodim07.png", "../data/tire.png"]
for image_path in image_paths:
# for image_path in visible_diff:
    print(image_path)
    image = read_image(image_path)


../data/1344829.png
../data/1349488.jpeg
../data/1350497.png
../data/1351987.jpeg
../data/1354206.jpeg
../data/43600825d9abb54db2aeeadadecf463b.jpg
../data/987919.jpg
../data/dark.png
../data/kodim01.png
../data/kodim05.jpg
../data/kodim07.png
../data/kodim21.png
../data/nikita-pishchugin-KDQ1BH-I3q4-unsplash.jpg
../data/sea.jpg
../data/tire.png
../data/vivek-7NgbMrUB60A-unsplash.jpg
